In [17]:
import pandas as pd
import glob
import numpy as np

from collections import defaultdict

# Datasets

In [16]:
def create_df_list(path):
    
    # get a list of all CSV files in the directory
    files = glob.glob(path + '/*.csv')

    # create an empty list to store the DataFrames
    df_list = []

    # iterate over the list of files
    for file in files:
        # read in the CSV file using pandas
        df = pd.read_csv(file)
        # append the DataFrame to the list
        df_list.append(df)

    # returns the list of dataframes
    return df_list

# Programm

## FD Finder

In [18]:
def fastFD(df, null_attributes):
    """
    Finds functional dependencies in a DataFrame using the FastFD algorithm.
    """
    # Step 1: Initialize the set of FDs with the trivial ones
    fds = set([(frozenset([attr]), frozenset([])) for attr in df.columns])
    non_trivial_fds = set()

    # Step 2: Compute the equivalence classes of the tuples
    eq_classes = compute_equivalence_classes(df)

    # Step 3: Compute the closure of each attribute set
    closures = {}
    for attr in df.columns:
        closures[frozenset([attr])] = closure(set([attr]), fds)

    # Step 4: Initialize the set of candidate pairs
    candidate_pairs = set([(frozenset([A]), frozenset([B])) for A in df.columns for B in list(null_attributes) if A != B])

    # Step 5: Repeat until there are no more FDs to be found
    while len(candidate_pairs) > 0:
        # Step 5a: Choose a pair of attributes (A, B) such that A -> B is not already known
        (A, B) = candidate_pairs.pop()

        # Step 5b: Compute A+
        AB_union = A.union(B)
        AB_closure = closures[A].intersection(closures[B])
        for attr in df.columns:
            AB_attr = AB_union.union(set([attr]))
            if AB_attr not in closures:
                closures[AB_attr] = closure(AB_attr, fds)

        # Step 5c: If B is in A+, add A -> B to the set of known FDs
        if B not in AB_closure:
            fds.add((A, B))
            non_trivial_fds.add((A, B))

            # Update the closures of all supersets of A
            for attr_set in [s for s in closures.keys() if A.issubset(s) and s != A]:
                closures[attr_set] = closures[attr_set].intersection(AB_closure)

            # Update the set of candidate pairs
            for C in null_attributes:
                if C not in AB_union:
                    candidate_pairs.add((AB_union, frozenset([C])))

    # Step 6: Return the set of known FDs
    result = [(list(X), list(Y)) for (X, Y) in fds if Y and len(X) <= 4]
    return result

def compute_equivalence_classes(df):
    """
    Computes the equivalence classes of the tuples in the DataFrame.
    """
    eq_classes = defaultdict(set)
    for row in df.itertuples(index=False):
        eq_classes[row] = set(row)
    return eq_classes

def closure(X, fds):
    """
    Computes the closure of a set of attributes X given a set of FDs.
    """
    X_closure = set(X)
    while True:
        changed = False
        for (A, B) in fds:
            if A.issubset(X_closure) and not B.issubset(X_closure):
                X_closure = X_closure.union(B)
                changed = True
        if not changed:
            break
    return X_closure

In [19]:
def verify_FDs(df, candidates):
    # create an empty list to store the verified functional dependencies
    fds = []

    # iterate over the list of candidate pairs of attributes
    for lhs, rhs in candidates:
        # group the DataFrame by the LHS attributes and check if the number of unique values in the RHS column is equal to 1
        if (df.groupby(list(lhs))[list(rhs)].nunique().eq(1).all() == True).all():
            # if the above condition is True for all groups, 
            # add the candidate pair to the list of verified functional dependencies
            fds.append((set(lhs), set(rhs)))

    # Remove any candidate pairs where there is another candidate pair with the same RHS and a larger LHS
    # create an empty list to store the pruned functional dependencies
    pruned_fds = []
    for lhs1, rhs1 in fds:
        # iterate over the list of verified functional dependencies and check if the RHS and LHS attributes of the current 
        # dependency are the same as those of another dependency, but the LHS of the other dependency is a proper subset of 
        # the LHS of the current dependency
        is_superset = False
        for lhs2, rhs2 in fds:
            if lhs1.issuperset(lhs2) and rhs1 == rhs2 and lhs1 != lhs2:
                is_superset = True
                break
        # if there is no other dependency with the same RHS and a larger LHS, add the current dependency to the list of pruned 
        # functional dependencies
        if not is_superset:
            pruned_fds.append((lhs1, rhs1))

    # return the list of pruned functional dependencies
    return pruned_fds


In [20]:
def find_FDs(df):
    # Find columns with null values
    null_attributes = df.columns[df.isnull().any()].tolist()
    
    # Generate candidate FDs using fastFD algorithm with the null attributes
    candidates = fastFD(df, null_attributes)
    
    # Verify the candidate FDs using the verify_FDs function
    verified_fds = verify_FDs(df, candidates)
    
    # If there are verified FDs, return them. Otherwise, return an empty list.
    if verified_fds != []:
        return verified_fds
    else:
        return []


## Null Replacer

In [21]:
def replace_Null(df, fds):
    # Iterate through each functional dependency (FD) in the list of verified FDs
    for lhs, rhs in fds:
        # Create copies of the LHS and RHS sets to modify
        rhs_copy = rhs.copy()
        lhs_copy = lhs.copy()
        
        # Check if the RHS set is not empty
        if len(rhs_copy) > 0:
            # Pop the last column name from the RHS set and the corresponding LHS column name
            rhs_col = rhs_copy.pop()
            lhs_col = lhs_copy.pop()

            # Step 1: Find index of rows with missing values in the RHS column
            idx = df.index[df[rhs_col].isnull()]

            # Step 2: Get corresponding LHS values for the rows with missing values in the RHS column
            lhs_values = df.loc[idx, lhs_col].tolist()

            # Step 3: Group the DataFrame by the LHS column
            grouped = df.groupby(lhs_col)

            # Step 4: Access the group of rows with the same LHS value(s) as the rows with missing values in the RHS column
            for lhs_value in lhs_values:
                # Check if the LHS value NaN
                if pd.isna(lhs_value):
                    continue
                else:
                    group = grouped.get_group(lhs_value)

                # Step 5: Get corresponding RHS value(s) of the rows in the group
                rhs_values = group[rhs_col].tolist()

                # Step 6: Replace the missing values in the RHS column with the corresponding value from the group with the same LHS value
                for v in rhs_values:
                    # Check if the RHS value is None or NaN
                    if pd.isna(v):
                        continue
                    else:
                        value = v
                        for i in idx:
                            if df.loc[i, lhs_col] == lhs_value:
                                df.loc[i, rhs_col] = value
                            else:
                                continue 
                    break
    return df


In [22]:
def replacer(df):
    # find the Functional Dependencies in the dataset
    fds = find_FDs(df)
    
    # count the number of null values before the replacement
    null_befor = df.isnull().sum().sum()

    # if there are no Functional Dependencies, return a message saying so
    if fds == []:
        return 'There are no Functional Dependencies.'
    else:
        # replace the null values in the dataset using the Functional Dependencies
        replace_Null(df, fds)
        
        # count the number of null values after the replacement
        null_after = df.isnull().sum().sum()
        
        # calculate the Replacement Rate as percentage of null values that were replaced
        if null_befor == 0:
            replace_rate = 0
        else:
            replace_rate = round((1-(null_after / null_befor)) * 100, 2)
    
    # return the updated dataset, the Replacement Rate, 
    # the initial number of null values, and the final number of null values
    return df, replace_rate, null_befor, null_after


## Main

In [23]:
def main(df, null_values):
    # replaces all values within the table that are in null-values witht NaN
    df.replace(null_values, np.nan, inplace=True)

    # Check if there are any null values in the DataFrame
    if df.isnull().any().any():
        # Call the replacer function to replace null values with FDs
        replacer2 = replacer(df)
        return replacer2
    else:
        return "There are no NULL-Values"


In [25]:
def dataset_iterator(df_list, null_values):
    replace_values = []
    # Iterate over each dataframe in the list
    for i in range(len(df_list)):
        df = df_list[i]
        # Call the main function to replace null values
        rate = main(df, null_values)
        # If there are no null values in the dataframe
        if isinstance(rate, str):
            # Add a string indicating there were no null values
            replace_values.append([i, rate])
        # If there were null values in the dataframe
        else:
            # Add the index of the dataframe and the replacement rate to the list
            replace_values.append([i, rate[1], rate[2], rate[3]])
    # Return the total number of dataframes and the list of replacement values
    return(len(df_list), replace_values)

null_values = ['None', '--', 'NaN', 'Null', 'NA', 'undefined', 'Inf', 'inf', 'NULL']
df_list = create_df_list("C:/Users/ilove/OneDrive/Uni/Master - Philipps Uni/3. Semester/Seminar - Lakehouse/Code/FDs_and_Null-Values/Used Datasets")
dataset_iterator(df_list, null_values)

# Evaluation

In [ ]:
# for evaluation porpus only

# Define a list of possible null values
null_values = ['None', '--', 'NaN', 'Null', 'NA', 'undefined', 'Inf', 'inf', 'NULL']

# Create a list of dataframes from a directory and perform data cleaning on each dataframe
# df_list = greate_df_list("C:/Users/ilove/Downloads/result_null_filtered/result_nan_filtered/Neuer Ordner")
# evaluation = dataset_iterator(df_list, null_values, 3)

# Initialize counters and a list to store the replacement rate for each dataframe
rate_list = []
counter_null = 0
counter_FD = 0
counter_rate = 0

# Iterate over the evaluation results for each dataframe
for j in range(len(evaluation[1])):
    if len(evaluation[1][j]) == 2:
        # If there are no null values in the dataframe, increment the counter
        if evaluation[1][j][1] == 'There are no NULL-Values':
            counter_null += 1
        # If there are FDs and null values, increment the counter and store the replacement rate
        else:
            counter_FD += 1
    else:
        # If there are null values but no FDs, increment the counter and store the replacement rate
        counter_rate += 1
        rate_list.append(evaluation[1][j])


count = [evaluation[0], counter_rate, counter_null, counter_FD, running_time]

# Create a pandas dataframe to store the replacement rates for each dataframe
df_rate = pd.DataFrame(rate_list, columns=['ID', 'replacement rate', 'Null befor', 'Null after'])
df_rate["Null difference"] = df_rate["Null befor"] - df_rate["Null after"]

# Export the results to a CSV file
# df_rate.to_csv('evaluation_Dataset.csv', index=False)
